In [188]:
import pandas as pd
import numpy as np
import datetime
from datetime import date  
import time

In [189]:
off_train = pd.read_csv('ccf_offline_stage1_train.csv')
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']
off_test = pd.read_csv('ccf_offline_stage1_test_revised.csv')
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']
#on_train = pd.read_csv('ccf_online_stage1_train.csv')
#on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']
off_test.head()
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [190]:
off_train.distance=off_train[["distance"]].fillna(-1)
off_test.distance=off_train[["distance"]].fillna(-1)

In [191]:
off_train.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,4663,11002.0,150:20,1.0,20160528.0,NaN
2,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
3,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613.0,NaN


In [192]:
#注意，比较时间的时候要类型一致，将原数据框中的数据改成float型
off_train.date=off_train.iloc[:,-1].astype("float")

In [193]:
#滑窗法划分，在这段时间内消费，或者是没消费但是收到券
dataset3 = off_test
feature3 = off_train[((off_train.date>=20160315)&(off_train.date<=20160630)|((off_train.date=='null')&(off_train.date_received>=20160315)&(off_train.date_received<=20160630)))]
dataset2 = off_train[(off_train.date_received>=20160515)&(off_train.date_received<=20160615)]
feature2 = off_train[(off_train.date>=20160201)&(off_train.date<=20160514)|((off_train.date=='null')&(off_train.date_received>=20160201)&(off_train.date_received<=20160514))]
dataset1 = off_train[(off_train.date_received>=20160414)&(off_train.date_received<=20160514)]
feature1 = off_train[(off_train.date>=20160101)&(off_train.date<=20160413)|((off_train.date=='null')&(off_train.date_received>=20160101)&(off_train.date_received<=20160413))]
dataset1.head(5)


E:\anaconda\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
7,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
18,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
43,4061024,3381,7610.0,200:20,10.0,20160426.0,NaN
52,106443,450,3732.0,30:5,-1.0,20160429.0,NaN
96,114747,1569,5054.0,200:30,9.0,20160426.0,NaN


In [194]:
print(dataset3.shape,dataset2.shape,dataset1.shape,feature3.shape,feature2.shape,feature1.shape)  #

(113640, 6) (258446, 7) (137167, 7) (554179, 7) (425292, 7) (375384, 7)


# other feature

other feature:
      this_month_user_receive_all_coupon_count
      this_month_user_receive_same_coupon_count
      this_month_user_receive_same_coupon_lastone
      this_month_user_receive_same_coupon_firstone
      this_day_user_receive_all_coupon_count
      this_day_user_receive_same_coupon_count
      day_gap_before, day_gap_after  (receive the same coupon)


# for dataset3

In [195]:
dataset3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received
0,4129537,450,9983,30:5,0.0,20160712
1,6949378,1300,3429,30:5,1.0,20160706
2,2166529,7113,6928,200:20,0.0,20160727
3,2166529,7113,1808,100:10,0.0,20160727
4,6172162,7605,6500,30:1,0.0,20160708


In [196]:
#注意在数据框中添加新的一列方法
# 用户收到的优惠券总和
t = dataset3[:][['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()

In [197]:
# 用户收到特定优惠券的总和
t1 = dataset3[:][['user_id','coupon_id']]
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()

In [198]:
# 用户此次之前或之后领使用优惠券的时间
t2 = dataset3[:][['user_id','coupon_id','date_received']]
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
t2.head()

,user_id,coupon_id,max_date_received,min_date_received
6,448,10927,20160710,20160704
13,736,3686,20160724,20160713
24,1318,11894,20160728,20160720
128,9208,6335,20160716,20160703
155,11170,11873,20160730,20160715


In [199]:
#只保留了最近接收时间和最远接受时间
t3 = dataset3[:][['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')    #右联接才有值
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received - t3.min_date_received
t3.head()

,user_id,coupon_id,date_received,max_date_received,min_date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,4129537,9983,20160712,NaN,NaN,NaN,NaN
1,6949378,3429,20160706,NaN,NaN,NaN,NaN
2,2166529,6928,20160727,NaN,NaN,NaN,NaN
3,2166529,1808,20160727,NaN,NaN,NaN,NaN
4,6172162,6500,20160708,NaN,NaN,NaN,NaN


In [200]:
# 只接受过一次优惠券为者为 -1
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [201]:
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]
t3.head()

,user_id,coupon_id,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,4129537,9983,20160712,-1,-1
1,6949378,3429,20160706,-1,-1
2,2166529,6928,20160727,-1,-1
3,2166529,1808,20160727,-1,-1
4,6172162,6500,20160708,-1,-1


In [202]:
# 第四个特征,一个用户所接收到的所有优惠券的数量
t4 = dataset3[:][['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
t4.head()

,user_id,date_received,this_day_user_receive_all_coupon_count
0,209,20160721,2
1,215,20160703,1
2,316,20160721,1
3,417,20160712,1
4,432,20160706,1


In [203]:
# 提取第五个特征,一个用户不同时间所接收到不同优惠券的数量
t5 = dataset3[:][['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
t5.head()

,user_id,coupon_id,date_received,this_day_user_receive_same_coupon_count
0,209,825,20160721,1
1,209,7557,20160721,1
2,215,5488,20160703,1
3,316,3992,20160721,1
4,417,12465,20160712,1


In [204]:
# 一个用户不同优惠券 的接受时间
t6 = dataset3[:][['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)   
t6.head()

,user_id,coupon_id,dates
0,209,825,20160721
1,209,7557,20160721
2,215,5488,20160703
3,316,3992,20160721
4,417,12465,20160712


In [205]:
# 接收优惠券最近的日子天数
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
 # 接收优惠券最远的日子天数       
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    




In [206]:
t7 = dataset3[:][['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
t7.head()

,user_id,coupon_id,date_received,day_gap_before,day_gap_after
0,4129537,9983,20160712,-1,-1
1,6949378,3429,20160706,-1,-1
2,2166529,6928,20160727,-1,-1
3,2166529,1808,20160727,-1,-1
4,6172162,6500,20160708,-1,-1


In [207]:
other_feature3 = pd.merge(t1,t,on='user_id')
other_feature3 = pd.merge(other_feature3,t3,on=['user_id','coupon_id'])
other_feature3 = pd.merge(other_feature3,t4,on=['user_id','date_received'])
other_feature3 = pd.merge(other_feature3,t5,on=['user_id','coupon_id','date_received'])
other_feature3 = pd.merge(other_feature3,t7,on=['user_id','coupon_id','date_received'])
other_feature3.to_csv('other_feature3.csv',index=None)
other_feature3.shape

(116204, 11)

# for dataset2

In [208]:
#把date_received变成int型，去掉了后面的.0,方便后续操作
dataset2.date_received=dataset2.iloc[:,-2].astype('int')
#dataset2.coupon_id=dataset2.iloc[:,-1].astype('int')
dataset2.head()

E:\anaconda\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
1,1439408,4663,11002.0,150:20,1.0,20160528,NaN
4,1439408,2632,8591.0,20:1,0.0,20160613,NaN
6,1439408,2632,8591.0,20:1,0.0,20160516,20160613.0
9,2029232,450,1532.0,30:5,0.0,20160530,NaN
10,2029232,6459,12737.0,20:1,0.0,20160519,NaN


In [209]:
t = dataset2[:][['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
t.head(5)

,user_id,this_month_user_receive_all_coupon_count
0,4,1
1,165,1
2,166,1
3,215,1
4,236,1


In [210]:
t1 = dataset2[:][['user_id','coupon_id']]
t1.coupon_id=t1.iloc[:,-1].astype('int')
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
t1.head()

,user_id,coupon_id,this_month_user_receive_same_coupon_count
0,4,2902,1
1,165,7571,1
2,166,9261,1
3,215,8944,1
4,236,11002,1


In [211]:
t2 = dataset2[:][['user_id','coupon_id','date_received']]
t2.coupon_id=t2.iloc[:,-2].astype('int')
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
t2.head()

,user_id,coupon_id,max_date_received,min_date_received
18,696,7571,20160524,20160520
31,947,12414,20160610,20160609
40,1318,2810,20160527,20160518
71,2266,4727,20160612,20160518
183,6504,6948,20160524,20160517


In [212]:
t3 = dataset2[['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received.astype('int')
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.astype('int') - t3.min_date_received
t3.head()

,user_id,coupon_id,date_received,max_date_received,min_date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,1439408,11002.0,20160528,NaN,NaN,NaN,NaN
1,1439408,8591.0,20160613,20160613.0,20160516.0,0.0,97.0
2,1439408,8591.0,20160516,20160613.0,20160516.0,97.0,0.0
3,2029232,1532.0,20160530,NaN,NaN,NaN,NaN
4,2029232,12737.0,20160519,NaN,NaN,NaN,NaN


In [213]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [214]:
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]
t3.head()

,user_id,coupon_id,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,1439408,11002.0,20160528,-1,-1
1,1439408,8591.0,20160613,1,0
2,1439408,8591.0,20160516,0,1
3,2029232,1532.0,20160530,-1,-1
4,2029232,12737.0,20160519,-1,-1


In [215]:

t4 = dataset2[:][['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
t4.head()

,user_id,date_received,this_day_user_receive_all_coupon_count
0,4,20160607,1
1,165,20160525,1
2,166,20160525,1
3,215,20160524,1
4,236,20160528,1


In [216]:
t5 = dataset2[:][['user_id','coupon_id','date_received']]
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
t5.head()

,user_id,coupon_id,date_received,this_day_user_receive_same_coupon_count
0,4,2902.0,20160607,1
1,165,7571.0,20160525,1
2,166,9261.0,20160525,1
3,215,8944.0,20160524,1
4,236,11002.0,20160528,1


In [217]:
t6 = dataset2[:][['user_id','coupon_id','date_received']]
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)
t6.head()

,user_id,coupon_id,dates
0,4,2902.0,20160607
1,165,7571.0,20160525
2,166,9261.0,20160525
3,215,8944.0,20160524
4,236,11002.0,20160528


In [218]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
    

In [219]:
t7 = dataset2[['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
t7.head()

,user_id,coupon_id,date_received,day_gap_before,day_gap_after
0,1439408,11002.0,20160528,-1,-1
1,1439408,8591.0,20160613,28,-1
2,1439408,8591.0,20160516,-1,28
3,2029232,1532.0,20160530,-1,-1
4,2029232,12737.0,20160519,-1,-1


In [220]:
other_feature2 = pd.merge(t1,t,on='user_id')
other_feature2 = pd.merge(other_feature2,t3,on=['user_id','coupon_id'])
other_feature2 = pd.merge(other_feature2,t4,on=['user_id','date_received'])
other_feature2 = pd.merge(other_feature2,t5,on=['user_id','coupon_id','date_received'])
other_feature2 = pd.merge(other_feature2,t7,on=['user_id','coupon_id','date_received'])
other_feature2.to_csv('other_feature2.csv',index=None)
other_feature2.shape

(262240, 11)

# for dataset1

In [221]:
#把date_received变成int型，去掉了后面的.0,方便后续操作
dataset1.date_received=dataset1.iloc[:,-2].astype('int')
dataset1.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
7,1832624,3381,7610.0,200:20,0.0,20160429,NaN
18,163606,1569,5054.0,200:30,10.0,20160421,NaN
43,4061024,3381,7610.0,200:20,10.0,20160426,NaN
52,106443,450,3732.0,30:5,-1.0,20160429,NaN
96,114747,1569,5054.0,200:30,9.0,20160426,NaN


In [222]:
t = dataset1[:][['user_id']]
t['this_month_user_receive_all_coupon_count'] = 1
t = t.groupby('user_id').agg('sum').reset_index()
t.head()

,user_id,this_month_user_receive_all_coupon_count
0,173,1
1,285,1
2,316,1
3,377,1
4,387,1


In [223]:
t1 = dataset1[:][['user_id','coupon_id']]
t1.coupon_id=t1.iloc[:,-1].astype('int')
t1['this_month_user_receive_same_coupon_count'] = 1
t1 = t1.groupby(['user_id','coupon_id']).agg('sum').reset_index()
t1.head()

,user_id,coupon_id,this_month_user_receive_same_coupon_count
0,173,7610,1
1,285,1532,1
2,316,8952,1
3,377,2857,1
4,387,7610,1


In [224]:
t2 = dataset1[:][['user_id','coupon_id','date_received']]
t2.coupon_id=t2.iloc[:,-1].astype('int')
t2.date_received = t2.date_received.astype('str')
t2 = t2.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t2['receive_number'] = t2.date_received.apply(lambda s:len(s.split(':')))
t2 = t2[t2.receive_number>1]
t2['max_date_received'] = t2.date_received.apply(lambda s:max([int(d) for d in s.split(':')]))
t2['min_date_received'] = t2.date_received.apply(lambda s:min([int(d) for d in s.split(':')]))
t2 = t2[['user_id','coupon_id','max_date_received','min_date_received']]
t2.head()

,user_id,coupon_id,max_date_received,min_date_received
25,1399,20160511,20160511,20160511
48,3057,20160508,20160508,20160508
60,3751,20160419,20160419,20160419
79,5047,20160513,20160513,20160513
84,5384,20160417,20160417,20160417


In [225]:
t3 = dataset1[:][['user_id','coupon_id','date_received']]
t3 = pd.merge(t3,t2,on=['user_id','coupon_id'],how='left')   #不知道这里应该是left 还是right
t3['this_month_user_receive_same_coupon_lastone'] = t3.max_date_received - t3.date_received.astype('int')
t3['this_month_user_receive_same_coupon_firstone'] = t3.date_received.astype('int') - t3.min_date_received
t3.head()

,user_id,coupon_id,date_received,max_date_received,min_date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,1832624,7610.0,20160429,NaN,NaN,NaN,NaN
1,163606,5054.0,20160421,NaN,NaN,NaN,NaN
2,4061024,7610.0,20160426,NaN,NaN,NaN,NaN
3,106443,3732.0,20160429,NaN,NaN,NaN,NaN
4,114747,5054.0,20160426,NaN,NaN,NaN,NaN


In [226]:
def is_firstlastone(x):
    if x==0:
        return 1
    elif x>0:
        return 0
    else:
        return -1 #those only receive once

In [227]:
t3.this_month_user_receive_same_coupon_lastone = t3.this_month_user_receive_same_coupon_lastone.apply(is_firstlastone)
t3.this_month_user_receive_same_coupon_firstone = t3.this_month_user_receive_same_coupon_firstone.apply(is_firstlastone)
t3 = t3[['user_id','coupon_id','date_received','this_month_user_receive_same_coupon_lastone','this_month_user_receive_same_coupon_firstone']]
t3.head()

,user_id,coupon_id,date_received,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone
0,1832624,7610.0,20160429,-1,-1
1,163606,5054.0,20160421,-1,-1
2,4061024,7610.0,20160426,-1,-1
3,106443,3732.0,20160429,-1,-1
4,114747,5054.0,20160426,-1,-1


In [228]:
t4 = dataset1[:][['user_id','date_received']]
t4['this_day_user_receive_all_coupon_count'] = 1
t4 = t4.groupby(['user_id','date_received']).agg('sum').reset_index()
t4.head()

,user_id,date_received,this_day_user_receive_all_coupon_count
0,173,20160414,1
1,285,20160501,1
2,316,20160430,1
3,377,20160512,1
4,387,20160421,1


In [229]:
t5 = dataset1[:][['user_id','coupon_id','date_received']]
t5.coupon_id=t5.iloc[:,-2].astype('int')
t5['this_day_user_receive_same_coupon_count'] = 1
t5 = t5.groupby(['user_id','coupon_id','date_received']).agg('sum').reset_index()
t5.head()

,user_id,coupon_id,date_received,this_day_user_receive_same_coupon_count
0,173,7610,20160414,1
1,285,1532,20160501,1
2,316,8952,20160430,1
3,377,2857,20160512,1
4,387,7610,20160421,1


In [230]:
t6 = dataset1[:][['user_id','coupon_id','date_received']]
t6.coupon_id=t6.iloc[:,-2].astype('int')
t6.date_received = t6.date_received.astype('str')
t6 = t6.groupby(['user_id','coupon_id'])['date_received'].agg(lambda x:':'.join(x)).reset_index()
t6.rename(columns={'date_received':'dates'},inplace=True)
t6.head()

,user_id,coupon_id,dates
0,173,7610,20160414
1,285,1532,20160501
2,316,8952,20160430
3,377,2857,20160512
4,387,7610,20160421


In [231]:
def get_day_gap_before(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))-date(int(d[0:4]),int(d[4:6]),int(d[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)
        
def get_day_gap_after(s):
    date_received,dates = s.split('-')
    dates = dates.split(':')
    gaps = []
    for d in dates:
        this_gap = (date(int(d[0:4]),int(d[4:6]),int(d[6:8]))-date(int(date_received[0:4]),int(date_received[4:6]),int(date_received[6:8]))).days
        if this_gap>0:
            gaps.append(this_gap)
    if len(gaps)==0:
        return -1
    else:
        return min(gaps)

In [232]:
t7 = dataset1[:][['user_id','coupon_id','date_received']]
t7 = pd.merge(t7,t6,on=['user_id','coupon_id'],how='left')
t7['date_received_date'] = t7.date_received.astype('str') + '-' + t7.dates
t7['day_gap_before'] = t7.date_received_date.apply(get_day_gap_before)
t7['day_gap_after'] = t7.date_received_date.apply(get_day_gap_after)
t7 = t7[['user_id','coupon_id','date_received','day_gap_before','day_gap_after']]
t7.head()

,user_id,coupon_id,date_received,day_gap_before,day_gap_after
0,1832624,7610.0,20160429,-1,-1
1,163606,5054.0,20160421,-1,-1
2,4061024,7610.0,20160426,-1,-1
3,106443,3732.0,20160429,-1,-1
4,114747,5054.0,20160426,-1,-1


In [233]:
other_feature1 = pd.merge(t1,t,on='user_id')
other_feature1 = pd.merge(other_feature1,t3,on=['user_id','coupon_id'])
other_feature1 = pd.merge(other_feature1,t4,on=['user_id','date_received'])
other_feature1 = pd.merge(other_feature1,t5,on=['user_id','coupon_id','date_received'])
other_feature1 = pd.merge(other_feature1,t7,on=['user_id','coupon_id','date_received'])
other_feature1.to_csv('other_feature1.csv',index=None)
other_feature1.shape

(139785, 11)

# coupon related feature   #############

(date_received) discount_rate.   discount_man.   discount_jian.    is_man_jian.    day_of_week.  day_of_month. 

In [234]:
def calc_discount_rate(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return float(s[0])
    else:
        return 1.0-float(s[1])/float(s[0])

def get_discount_man(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[0])
        
def get_discount_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 'null'
    else:
        return int(s[1])

def is_man_jian(s):
    s =str(s)
    s = s.split(':')
    if len(s)==1:
        return 0
    else:
        return 1

# dataset3

In [235]:
dataset3['day_of_week'] = dataset3.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset3['day_of_month'] = dataset3.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset3['days_distance'] = dataset3.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,6,30)).days)
dataset3['discount_man'] = dataset3.discount_rate.apply(get_discount_man)
dataset3['discount_jian'] = dataset3.discount_rate.apply(get_discount_jian)
dataset3['is_man_jian'] = dataset3.discount_rate.apply(is_man_jian)
dataset3['discount_rate'] = dataset3.discount_rate.apply(calc_discount_rate)
d = dataset3[:][['coupon_id']]
d['coupon_count'] = 1
# d = dataset3[['user_id','coupon_id']]   #在这里插入按'user_id','coupon_id' 进行计数，而不是仅对coupon_id计数
# col_name = list(d.columns)
# col_name.insert(2,'coupon_count')
# d.reindex(columns = col_name,fill_value = 1)
#d = d.groupby(['user_id','coupon_id']).agg('sum').reset_index
d = d.groupby('coupon_id').agg('sum').reset_index()
#dataset3 = pd.merge(dataset3,d,on=['user_id','coupon_id'],how='left')
dataset3 = pd.merge(dataset3,d,on='coupon_id',how='left')
dataset3.to_csv('coupon3_feature.csv',index=None)
dataset3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,4129537,450,9983,0.833333,0.0,20160712,2,12,12,30,5,1,11586
1,6949378,1300,3429,0.833333,1.0,20160706,3,6,6,30,5,1,4887
2,2166529,7113,6928,0.900000,0.0,20160727,3,27,27,200,20,1,205
3,2166529,7113,1808,0.900000,0.0,20160727,3,27,27,100,10,1,205
4,6172162,7605,6500,0.966667,0.0,20160708,5,8,8,30,1,1,132


In [236]:
dataset3.shape

(113640, 13)

# dataset2

In [237]:

dataset2['day_of_week'] = dataset2.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset2['day_of_month'] = dataset2.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset2['days_distance'] = dataset2.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,5,14)).days)
dataset2['discount_man'] = dataset2.discount_rate.apply(get_discount_man)
dataset2['discount_jian'] = dataset2.discount_rate.apply(get_discount_jian)
dataset2['is_man_jian'] = dataset2.discount_rate.apply(is_man_jian)
dataset2['discount_rate'] = dataset2.discount_rate.apply(calc_discount_rate)

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

In [238]:

d = dataset2[:][['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset2 = pd.merge(dataset2,d,on='coupon_id',how='left')
dataset2.to_csv('coupon2_feature.csv',index=None)
dataset2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1439408,4663,11002.0,0.866667,1.0,20160528,NaN,6,28,14,150,20,1,7730
1,1439408,2632,8591.0,0.950000,0.0,20160613,NaN,1,13,30,20,1,1,5
2,1439408,2632,8591.0,0.950000,0.0,20160516,20160613.0,1,16,2,20,1,1,5
3,2029232,450,1532.0,0.833333,0.0,20160530,NaN,1,30,16,30,5,1,11728
4,2029232,6459,12737.0,0.950000,0.0,20160519,NaN,4,19,5,20,1,1,16


In [239]:
dataset2.shape

(258446, 14)

# dataset1

In [240]:


dataset1['day_of_week'] = dataset1.date_received.astype('str').apply(lambda x:date(int(x[0:4]),int(x[4:6]),int(x[6:8])).weekday()+1)
dataset1['day_of_month'] = dataset1.date_received.astype('str').apply(lambda x:int(x[6:8]))
dataset1['days_distance'] = dataset1.date_received.astype('str').apply(lambda x:(date(int(x[0:4]),int(x[4:6]),int(x[6:8]))-date(2016,4,13)).days)
dataset1['discount_man'] = dataset1.discount_rate.apply(get_discount_man)
dataset1['discount_jian'] = dataset1.discount_rate.apply(get_discount_jian)
dataset1['is_man_jian'] = dataset1.discount_rate.apply(is_man_jian)
dataset1['discount_rate'] = dataset1.discount_rate.apply(calc_discount_rate)
d = dataset1[:][['coupon_id']]
d['coupon_count'] = 1
d = d.groupby('coupon_id').agg('sum').reset_index()
dataset1 = pd.merge(dataset1,d,on='coupon_id',how='left')
dataset1.to_csv('coupon1_feature.csv',index=None)
dataset1.head()

E:\anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
E:\anaconda\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
E:\anaconda\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count
0,1832624,3381,7610.0,0.900000,0.0,20160429,NaN,5,29,16,200,20,1,22281
1,163606,1569,5054.0,0.850000,10.0,20160421,NaN,4,21,8,200,30,1,11893
2,4061024,3381,7610.0,0.900000,10.0,20160426,NaN,2,26,13,200,20,1,22281
3,106443,450,3732.0,0.833333,-1.0,20160429,NaN,5,29,16,30,5,1,8845
4,114747,1569,5054.0,0.850000,9.0,20160426,NaN,2,26,13,200,30,1,11893


In [241]:
dataset1.shape


(137167, 14)

# ############# merchant related feature   #############

total_sales.   sales_use_coupon.    total_coupon
coupon_rate = sales_use_coupon/total_sales.  
transfer_rate = sales_use_coupon/total_coupon. 
merchant_avg_distance,    merchant_min_distance,    merchant_max_distance of those use coupon


# for dataset3

In [242]:

feature3.head()


,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0
11,2029232,6459,NaN,NaN,0.0,NaN,20160626.0
12,2029232,6459,NaN,NaN,0.0,NaN,20160519.0
14,196342,1579,NaN,NaN,1.0,NaN,20160606.0


In [243]:
merchant3 = feature3[['merchant_id','coupon_id','distance','date_received','date']]
merchant3  .head()

,merchant_id,coupon_id,distance,date_received,date
5,2632,NaN,0.0,NaN,20160516.0
6,2632,8591.0,0.0,20160516.0,20160613.0
11,6459,NaN,0.0,NaN,20160626.0
12,6459,NaN,0.0,NaN,20160519.0
14,1579,NaN,1.0,NaN,20160606.0


In [244]:
merchant3.shape

(554179, 5)

In [245]:
#对商户去重
t = merchant3[:][['merchant_id']]
t=t.drop_duplicates()
t.head()


,merchant_id
5,2632
11,6459
14,1579
21,8390
28,3051


In [246]:
t1 = merchant3[merchant3.date!='null'][['merchant_id']]
t1['total_sales'] = 1

# t1.merchant_id=t1['merchant_id'].apply(str)
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
t1.head()


,merchant_id,total_sales
0,1,10
1,2,4
2,3,9
3,4,29
4,5,43


In [247]:
t2 = merchant3[(merchant3.date!='null')&(merchant3.coupon_id!='null')][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()

In [248]:
# 提取商品优惠券的总数量
t3 = merchant3[merchant3.coupon_id!='null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
t3.head()

,merchant_id,total_coupon
0,1,10
1,2,4
2,3,9
3,4,29
4,5,43


In [249]:
# 把数据中的空值全部替换为 -1
t4 = merchant3[(merchant3.date!='null')&(merchant3.coupon_id!='null')][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.replace(np.nan,-1,inplace=True)
t4.distance = t4.distance.astype('int')
t4.head()

,merchant_id,distance
5,2632,0
6,2632,0
11,6459,0
12,6459,0
14,1579,1


In [250]:
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace=True)

t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace=True)

In [251]:
t5.head()

,merchant_id,merchant_min_distance
0,1,0
1,2,0
2,3,0
3,4,0
4,5,-1


In [252]:
merchant3_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t2,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t3,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t5,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t6,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t7,on='merchant_id',how='left')
merchant3_feature = pd.merge(merchant3_feature,t8,on='merchant_id',how='left')
merchant3_feature.sales_use_coupon = merchant3_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_coupon
merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon.astype('float') / merchant3_feature.total_sales
merchant3_feature.total_coupon = merchant3_feature.total_coupon.replace(np.nan,0) #fillna with 0
merchant3_feature.to_csv('merchant3_feature.csv',index=None)

In [253]:
merchant3_feature.head()

,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,2632,8,8,8,0,10,1.625000,0.5,1.0,1.0
1,6459,20,20,20,0,6,1.050000,0.0,1.0,1.0
2,1579,44,44,44,0,5,1.227273,1.0,1.0,1.0
3,8390,1726,1726,1726,-1,10,0.476246,0.0,1.0,1.0
4,3051,2,2,2,0,0,0.000000,0.0,1.0,1.0


In [254]:
merchant3_feature.shape

(8194, 10)

# for dataset2

In [255]:
merchant2 = feature2[:][['merchant_id','coupon_id','distance','date_received','date']]
merchant2.head()

,merchant_id,coupon_id,distance,date_received,date
0,2632,NaN,0.0,NaN,20160217.0
21,8390,NaN,0.0,NaN,20160327.0
38,8390,7531.0,0.0,20160321.0,20160329.0
40,8390,NaN,0.0,NaN,20160321.0
46,2934,NaN,0.0,NaN,20160322.0


In [256]:
t = merchant2[:][['merchant_id']]
t.drop_duplicates(inplace=True)

In [257]:
t1 = merchant2[merchant2.date!='null'][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
t1.head()

,merchant_id,total_sales
0,1,3
1,3,8
2,4,19
3,5,16
4,6,36


In [258]:
t2 = merchant2[(merchant2.date!='null')&(merchant2.coupon_id!='null')][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
t2.head()

,merchant_id,sales_use_coupon
0,1,3
1,3,8
2,4,19
3,5,16
4,6,36


In [259]:
t3 = merchant2[merchant2.coupon_id!='null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
t3.head()

,merchant_id,total_coupon
0,1,3
1,3,8
2,4,19
3,5,16
4,6,36


In [260]:
t4 = merchant2[(merchant2.date!='null')&(merchant2.coupon_id!='null')][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.replace(np.nan,-1,inplace=True)
t4.distance = t4.distance.astype('int')
t4.head()

,merchant_id,distance
0,2632,0
21,8390,0
38,8390,0
40,8390,0
46,2934,0


In [261]:
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace=True)
t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace=True)


In [262]:
t5.head()

,merchant_id,merchant_min_distance
0,1,0
1,3,0
2,4,0
3,5,0
4,6,0


In [263]:
merchant2_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t2,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t3,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t5,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t6,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t7,on='merchant_id',how='left')
merchant2_feature = pd.merge(merchant2_feature,t8,on='merchant_id',how='left')
merchant2_feature.sales_use_coupon = merchant2_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant2_feature['merchant_coupon_transfer_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_coupon
merchant2_feature['coupon_rate'] = merchant2_feature.sales_use_coupon.astype('float') / merchant2_feature.total_sales
merchant2_feature.total_coupon = merchant2_feature.total_coupon.replace(np.nan,0) #fillna with 0
merchant2_feature.to_csv('merchant2_feature.csv',index=None)

In [264]:
merchant2_feature.head()

,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,2632,14,14,14,0,10,2.071429,1.0,1.0,1.0
1,8390,1207,1207,1207,-1,10,0.463132,0.0,1.0,1.0
2,2934,21730,21730,21730,-1,10,0.906397,0.0,1.0,1.0
3,2454,341,341,341,-1,10,0.387097,0.0,1.0,1.0
4,5341,20891,20891,20891,-1,10,0.128764,0.0,1.0,1.0


In [265]:
merchant2_feature.shape

(7380, 10)

# for dataset1

In [266]:
merchant1 = feature1[:][['merchant_id','coupon_id','distance','date_received','date']]
merchant1.head()

,merchant_id,coupon_id,distance,date_received,date
0,2632,NaN,0.0,NaN,20160217.0
21,8390,NaN,0.0,NaN,20160327.0
26,7884,NaN,10.0,NaN,20160115.0
30,1041,NaN,2.0,NaN,20160114.0
38,8390,7531.0,0.0,20160321.0,20160329.0


In [267]:
t = merchant1[:][['merchant_id']]
t.drop_duplicates(inplace=True)
t.head()

,merchant_id
0,2632
21,8390
26,7884
30,1041
46,2934


In [268]:
t1 = merchant1[merchant1.date!='null'][['merchant_id']]
t1['total_sales'] = 1
t1 = t1.groupby('merchant_id').agg('sum').reset_index()
t1.head()

,merchant_id,total_sales
0,1,4
1,4,12
2,5,17
3,6,27
4,9,1


In [269]:

t2 = merchant1[(merchant1.date!='null')&(merchant1.coupon_id!='null')][['merchant_id']]
t2['sales_use_coupon'] = 1
t2 = t2.groupby('merchant_id').agg('sum').reset_index()
t2.head()

,merchant_id,sales_use_coupon
0,1,4
1,4,12
2,5,17
3,6,27
4,9,1


In [270]:

t3 = merchant1[merchant1.coupon_id!='null'][['merchant_id']]
t3['total_coupon'] = 1
t3 = t3.groupby('merchant_id').agg('sum').reset_index()
t3.head()

,merchant_id,total_coupon
0,1,4
1,4,12
2,5,17
3,6,27
4,9,1


In [271]:

t4 = merchant1[(merchant1.date!='null')&(merchant1.coupon_id!='null')][['merchant_id','distance']]
t4.replace('null',-1,inplace=True)
t4.replace(np.nan,-1,inplace=True)
t4.distance = t4.distance.astype('int')
t4.head()

,merchant_id,distance
0,2632,0
21,8390,0
26,7884,10
30,1041,2
38,8390,0


In [272]:
t5 = t4.groupby('merchant_id').agg('min').reset_index()
t5.rename(columns={'distance':'merchant_min_distance'},inplace=True)

t6 = t4.groupby('merchant_id').agg('max').reset_index()
t6.rename(columns={'distance':'merchant_max_distance'},inplace=True)

t7 = t4.groupby('merchant_id').agg('mean').reset_index()
t7.rename(columns={'distance':'merchant_mean_distance'},inplace=True)

t8 = t4.groupby('merchant_id').agg('median').reset_index()
t8.rename(columns={'distance':'merchant_median_distance'},inplace=True)


In [273]:
merchant1_feature = pd.merge(t,t1,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t2,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t3,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t5,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t6,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t7,on='merchant_id',how='left')
merchant1_feature = pd.merge(merchant1_feature,t8,on='merchant_id',how='left')
merchant1_feature.sales_use_coupon = merchant1_feature.sales_use_coupon.replace(np.nan,0) #fillna with 0
merchant1_feature['merchant_coupon_transfer_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_coupon
merchant1_feature['coupon_rate'] = merchant1_feature.sales_use_coupon.astype('float') / merchant1_feature.total_sales
merchant1_feature.total_coupon = merchant1_feature.total_coupon.replace(np.nan,0) #fillna with 0
merchant1_feature.to_csv('merchant1_feature.csv',index=None)

In [274]:
merchant1_feature.head()

,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate
0,2632,14,14,14,0,10,1.357143,1.0,1.0,1.0
1,8390,1056,1056,1056,-1,10,0.448864,0.0,1.0,1.0
2,7884,1464,1464,1464,-1,10,1.997951,0.0,1.0,1.0
3,1041,2078,2078,2078,-1,10,1.075553,0.0,1.0,1.0
4,2934,20717,20717,20717,-1,10,0.880533,0.0,1.0,1.0


In [275]:
merchant1_feature.shape

(5864, 10)

# #########user related feature   #############

3.user related: 
      count_merchant. 
      user_avg_distance, user_min_distance,user_max_distance. 
      buy_use_coupon. buy_total. coupon_received.
      buy_use_coupon/coupon_received. 
      buy_use_coupon/buy_total
      user_date_datereceived_gap
      

"""

In [276]:
def get_user_date_datereceived_gap(s):
    s = s.split(':')
    return (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8])) - date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days


# for dataset3

In [277]:
user3 = feature3[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
user3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0
11,2029232,6459,NaN,NaN,0.0,NaN,20160626.0
12,2029232,6459,NaN,NaN,0.0,NaN,20160519.0
14,196342,1579,NaN,NaN,1.0,NaN,20160606.0


In [278]:
t = user3[:][['user_id']]
t.drop_duplicates(inplace=True)
t.head()

,user_id
5,1439408
11,2029232
14,196342
21,253750
28,4191584


In [279]:
t1 = user3[user3.date!='null'][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
t1.head()

,user_id,count_merchant
0,165,2
1,209,1
2,239,1
3,285,1
4,315,2


In [280]:
t2 = user3[(user3.date!='null')&(user3.coupon_id!='null')][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.replace(np.nan,-1,inplace=True)
t2.distance = t2.distance.astype('int')
t2.head()

,user_id,distance
5,1439408,0
6,1439408,0
11,2029232,0
12,2029232,0
14,196342,1


In [281]:
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_distance'},inplace=True)
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)
t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)
t7 = user3[(user3.date!='null')&(user3.coupon_id!='null')][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
t8 = user3[user3.date!='null'][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
t9 = user3[user3.coupon_id!='null'][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()



In [282]:
user3.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
5,1439408,2632,NaN,NaN,0.0,NaN,20160516.0
6,1439408,2632,8591.0,20:1,0.0,20160516.0,20160613.0
11,2029232,6459,NaN,NaN,0.0,NaN,20160626.0
12,2029232,6459,NaN,NaN,0.0,NaN,20160519.0
14,196342,1579,NaN,NaN,1.0,NaN,20160606.0


In [283]:
t10 = user3[(user3.date_received!='null')&(user3.date!='null')][['user_id','date_received','date']]

t10=t10.where(t10.notnull(),'null')  #将NaN替换成null
t10 = t10[(t10.date_received!='null')][['user_id','date_received','date']]
t10.date_received=t10.iloc[:,-2].astype('int')

t10.date_received=t10['date_received'].apply(str)
t10.date=t10['date'].apply(str)                     #都变成字符型
t10['user_date_datereceived_gap'] = t10.date +':' +t10.date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
t10.head()

,user_id,user_date_datereceived_gap
6,1439408,28
33,1113008,6
38,2881376,8
69,114747,13
84,114747,7


In [284]:
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)
t13.head()

,user_id,max_user_date_datereceived_gap
0,417,14
1,687,5
2,696,6
3,947,1
4,1302,10


In [285]:
user3_feature = pd.merge(t,t1,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t3,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t4,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t5,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t6,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t7,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t8,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t9,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t11,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t12,on='user_id',how='left')
user3_feature = pd.merge(user3_feature,t13,on='user_id',how='left')
user3_feature.count_merchant = user3_feature.count_merchant.replace(np.nan,0)
user3_feature.buy_use_coupon = user3_feature.buy_use_coupon.replace(np.nan,0)
user3_feature['buy_use_coupon_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.buy_total.astype('float')
user3_feature['user_coupon_transfer_rate'] = user3_feature.buy_use_coupon.astype('float') / user3_feature.coupon_received.astype('float')
user3_feature.buy_total = user3_feature.buy_total.replace(np.nan,0)
user3_feature.coupon_received = user3_feature.coupon_received.replace(np.nan,0)
user3_feature.to_csv('user3_feature.csv',index=None)

In [286]:
user3_feature.head()

,user_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1,0,0,0.0,0.0,2,2,2,28.0,28.0,28.0,1.0,1.0
1,2029232,1,0,0,0.0,0.0,2,2,2,NaN,NaN,NaN,1.0,1.0
2,196342,1,1,1,1.0,1.0,1,1,1,NaN,NaN,NaN,1.0,1.0
3,253750,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
4,4191584,1,0,0,0.0,0.0,2,2,2,NaN,NaN,NaN,1.0,1.0


In [287]:
user3_feature.shape

(180561, 14)

# for dataset2

In [288]:
user2 = feature2[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
user2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
21,253750,8390,NaN,NaN,0.0,NaN,20160327.0
38,2881376,8390,7531.0,20:5,0.0,20160321.0,20160329.0
40,2881376,8390,NaN,NaN,0.0,NaN,20160321.0
46,7073472,2934,NaN,NaN,0.0,NaN,20160322.0


In [289]:
t = user2[:][['user_id']]
t.drop_duplicates(inplace=True)

In [290]:
t1 = user2[user2.date!='null'][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
t1.head()

,user_id,count_merchant
0,165,2
1,184,1
2,215,1
3,285,1
4,315,2


In [291]:
t2 = user2[(user2.date!='null')&(user2.coupon_id!='null')][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.replace(np.nan,-1,inplace=True)
t2.distance = t2.distance.astype('int')
t2.head()

,user_id,distance
0,1439408,0
21,253750,0
38,2881376,0
40,2881376,0
46,7073472,0


In [292]:
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_distance'},inplace=True)
t3.head()

,user_id,user_min_distance
0,165,0
1,184,0
2,215,1
3,285,2
4,315,1


In [293]:
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)
t4.head()

,user_id,user_max_distance
0,165,0
1,184,0
2,215,1
3,285,2
4,315,5


In [294]:
t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)

In [295]:
t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)

t7 = user2[(user2.date!='null')&(user2.coupon_id!='null')][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
t7.head()

,user_id,buy_use_coupon
0,165,8
1,184,1
2,215,1
3,285,1
4,315,3


In [296]:
t8 = user2[user2.date!='null'][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
t8.head()

,user_id,buy_total
0,165,8
1,184,1
2,215,1
3,285,1
4,315,3


In [297]:
t9 = user2[user2.coupon_id!='null'][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()

In [298]:
user2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
21,253750,8390,NaN,NaN,0.0,NaN,20160327.0
38,2881376,8390,7531.0,20:5,0.0,20160321.0,20160329.0
40,2881376,8390,NaN,NaN,0.0,NaN,20160321.0
46,7073472,2934,NaN,NaN,0.0,NaN,20160322.0


In [299]:
t10 = user2[(user2.date_received!='null')&(user2.date!='null')][['user_id','date_received','date']]

t10=t10.where(t10.notnull(),'null')  #将NaN替换成null
t10 = t10[(t10.date_received!='null')][['user_id','date_received','date']]
t10.date_received=t10.iloc[:,-2].astype('int')

t10.date_received=t10['date_received'].apply(str)
t10.date=t10['date'].apply(str)                     #都变成字符型
t10['user_date_datereceived_gap'] = t10.date +':' +t10.date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
t10.head()

,user_id,user_date_datereceived_gap
38,2881376,8
75,114747,25
76,114747,11
84,114747,7
100,114747,23


In [300]:
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)


In [301]:
user2_feature = pd.merge(t,t1,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t3,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t4,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t5,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t6,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t7,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t8,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t9,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t11,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t12,on='user_id',how='left')
user2_feature = pd.merge(user2_feature,t13,on='user_id',how='left')
user2_feature.count_merchant = user2_feature.count_merchant.replace(np.nan,0)
user2_feature.buy_use_coupon = user2_feature.buy_use_coupon.replace(np.nan,0)
user2_feature['buy_use_coupon_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.buy_total.astype('float')
user2_feature['user_coupon_transfer_rate'] = user2_feature.buy_use_coupon.astype('float') / user2_feature.coupon_received.astype('float')
user2_feature.buy_total = user2_feature.buy_total.replace(np.nan,0)
user2_feature.coupon_received = user2_feature.coupon_received.replace(np.nan,0)
user2_feature.to_csv('user2_feature.csv',index=None)



In [302]:
user2_feature.head()

,user_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
1,253750,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
2,2881376,1,0,0,0.0,0.0,2,2,2,8.0,8.0,8.0,1.0,1.0
3,7073472,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
4,114747,3,0,0,0.0,0.0,11,11,11,16.5,7.0,25.0,1.0,1.0


In [303]:
user2_feature.shape

(151988, 14)

# for dataset1

In [304]:
user1 = feature1[['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']]
user1.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
21,253750,8390,NaN,NaN,0.0,NaN,20160327.0
26,1964720,7884,NaN,NaN,10.0,NaN,20160115.0
30,1113008,1041,NaN,NaN,2.0,NaN,20160114.0
38,2881376,8390,7531.0,20:5,0.0,20160321.0,20160329.0


In [305]:
t = user1[:][['user_id']]
t.drop_duplicates(inplace=True)
t.head()

,user_id
0,1439408
21,253750
26,1964720
30,1113008
38,2881376


In [306]:
t1 = user1[user1.date!='null'][['user_id','merchant_id']]
t1.drop_duplicates(inplace=True)
t1.merchant_id = 1
t1 = t1.groupby('user_id').agg('sum').reset_index()
t1.rename(columns={'merchant_id':'count_merchant'},inplace=True)
t1.head()

,user_id,count_merchant
0,165,2
1,184,1
2,215,1
3,285,1
4,315,1


In [307]:
t2 = user1[(user1.date!='null')&(user1.coupon_id!='null')][['user_id','distance']]
t2.replace('null',-1,inplace=True)
t2.replace(np.nan,-1,inplace=True)
t2.distance = t2.distance.astype('int')
t3 = t2.groupby('user_id').agg('min').reset_index()
t3.rename(columns={'distance':'user_min_distance'},inplace=True)
t3.head()

,user_id,user_min_distance
0,165,0
1,184,0
2,215,1
3,285,2
4,315,1


In [308]:
t4 = t2.groupby('user_id').agg('max').reset_index()
t4.rename(columns={'distance':'user_max_distance'},inplace=True)

t5 = t2.groupby('user_id').agg('mean').reset_index()
t5.rename(columns={'distance':'user_mean_distance'},inplace=True)

t6 = t2.groupby('user_id').agg('median').reset_index()
t6.rename(columns={'distance':'user_median_distance'},inplace=True)

In [309]:
t7 = user1[(user1.date!='null')&(user1.coupon_id!='null')][['user_id']]
t7['buy_use_coupon'] = 1
t7 = t7.groupby('user_id').agg('sum').reset_index()
t8 = user1[user1.date!='null'][['user_id']]
t8['buy_total'] = 1
t8 = t8.groupby('user_id').agg('sum').reset_index()
t9 = user1[user1.coupon_id!='null'][['user_id']]
t9['coupon_received'] = 1
t9 = t9.groupby('user_id').agg('sum').reset_index()


In [310]:
t10 = user1[(user1.date_received!='null')&(user1.date!='null')][['user_id','date_received','date']]
t10=t10.where(t10.notnull(),'null')  #将NaN替换成null
t10 = t10[(t10.date_received!='null')][['user_id','date_received','date']]
t10.date_received=t10.iloc[:,-2].astype('int')

t10.date_received=t10['date_received'].apply(str)
t10.date=t10['date'].apply(str)                     #都变成字符型
t10['user_date_datereceived_gap'] = t10.date +':' +t10.date_received
t10.user_date_datereceived_gap = t10.user_date_datereceived_gap.apply(get_user_date_datereceived_gap)
t10 = t10[['user_id','user_date_datereceived_gap']]
t10.head()

,user_id,user_date_datereceived_gap
38,2881376,8
75,114747,25
76,114747,11
77,114747,1
84,114747,7


In [311]:
t11 = t10.groupby('user_id').agg('mean').reset_index()
t11.rename(columns={'user_date_datereceived_gap':'avg_user_date_datereceived_gap'},inplace=True)
t12 = t10.groupby('user_id').agg('min').reset_index()
t12.rename(columns={'user_date_datereceived_gap':'min_user_date_datereceived_gap'},inplace=True)
t13 = t10.groupby('user_id').agg('max').reset_index()
t13.rename(columns={'user_date_datereceived_gap':'max_user_date_datereceived_gap'},inplace=True)


In [312]:
user1_feature = pd.merge(t,t1,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t3,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t4,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t5,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t6,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t7,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t8,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t9,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t11,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t12,on='user_id',how='left')
user1_feature = pd.merge(user1_feature,t13,on='user_id',how='left')
user1_feature.count_merchant = user1_feature.count_merchant.replace(np.nan,0)
user1_feature.buy_use_coupon = user1_feature.buy_use_coupon.replace(np.nan,0)
user1_feature['buy_use_coupon_rate'] = user1_feature.buy_use_coupon.astype('float') / user1_feature.buy_total.astype('float')
user1_feature['user_coupon_transfer_rate'] = user1_feature.buy_use_coupon.astype('float') / user1_feature.coupon_received.astype('float')
user1_feature.buy_total = user1_feature.buy_total.replace(np.nan,0)
user1_feature.coupon_received = user1_feature.coupon_received.replace(np.nan,0)
user1_feature.to_csv('user1_feature.csv',index=None)


In [313]:
user1_feature.head()

,user_id,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate
0,1439408,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
1,253750,1,0,0,0.0,0.0,1,1,1,NaN,NaN,NaN,1.0,1.0
2,1964720,1,10,10,10.0,10.0,1,1,1,NaN,NaN,NaN,1.0,1.0
3,1113008,1,2,2,2.0,2.0,1,1,1,NaN,NaN,NaN,1.0,1.0
4,2881376,1,0,0,0.0,0.0,2,2,2,8.0,8.0,8.0,1.0,1.0


In [314]:
user1_feature.shape

(139229, 14)

# ##################  user_merchant related feature 

4.user_merchant:
      times_user_buy_merchant_before. 
"""

# for dataset3

In [315]:
all_user_merchant = feature3[:][['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)

In [316]:
t = feature3[:][['user_id','merchant_id','date']]
t = t[t.date!='null'][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)
t.head()


,user_id,merchant_id,user_merchant_buy_total
0,165,2934,6
1,165,4195,6
2,209,3267,1
3,239,3465,1
4,285,450,1


In [317]:
t1 = feature3[:][['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id!='null'][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace=True)
t1.head()

,user_id,merchant_id,user_merchant_received
0,165,2934,6
1,165,4195,6
2,209,3267,1
3,239,3465,1
4,285,450,1


In [318]:
t2 = feature3[:][['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!='null')&(t2.date_received!='null')][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)
t2.head()

,user_id,merchant_id,user_merchant_buy_use_coupon
0,165,2934,6
1,165,4195,6
2,209,3267,1
3,239,3465,1
4,285,450,1


In [319]:
t3 = feature3[:][['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
t3.head()

,user_id,merchant_id,user_merchant_any
0,165,2934,6
1,165,4195,6
2,209,3267,1
3,239,3465,1
4,285,450,1


In [320]:
t4 = feature3[:][['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date!='null')&(t4.coupon_id=='null')][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)
t4.head()

,user_id,merchant_id,user_merchant_buy_common


In [321]:
user_merchant3 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t1,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t2,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t3,on=['user_id','merchant_id'],how='left')
user_merchant3 = pd.merge(user_merchant3,t4,on=['user_id','merchant_id'],how='left')
user_merchant3.user_merchant_buy_use_coupon = user_merchant3.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant3.user_merchant_buy_common = user_merchant3.user_merchant_buy_common.replace(np.nan,0)
user_merchant3['user_merchant_coupon_transfer_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_received.astype('float')
user_merchant3['user_merchant_coupon_buy_rate'] = user_merchant3.user_merchant_buy_use_coupon.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3['user_merchant_rate'] = user_merchant3.user_merchant_buy_total.astype('float') / user_merchant3.user_merchant_any.astype('float')
user_merchant3['user_merchant_common_buy_rate'] = user_merchant3.user_merchant_buy_common.astype('float') / user_merchant3.user_merchant_buy_total.astype('float')
user_merchant3.to_csv('user_merchant3.csv',index=None)


In [322]:
user_merchant3.head()

,user_id,merchant_id,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate
0,1439408,2632,2,2,2,2,0.0,1.0,1.0,1.0,0.0
1,2029232,6459,2,2,2,2,0.0,1.0,1.0,1.0,0.0
2,196342,1579,1,1,1,1,0.0,1.0,1.0,1.0,0.0
3,253750,8390,1,1,1,1,0.0,1.0,1.0,1.0,0.0
4,4191584,3051,2,2,2,2,0.0,1.0,1.0,1.0,0.0


In [323]:
user_merchant3.shape

(230728, 11)

# for dataset2

In [324]:
all_user_merchant = feature2[:][['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)

In [325]:
t = feature2[:][['user_id','merchant_id','date']]
t = t[t.date!='null'][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)
t.head()

,user_id,merchant_id,user_merchant_buy_total
0,165,2934,4
1,165,4195,4
2,184,3381,1
3,215,129,1
4,285,450,1


In [326]:
t1 = feature2[:][['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id!='null'][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace=True)
t1.head()


,user_id,merchant_id,user_merchant_received
0,165,2934,4
1,165,4195,4
2,184,3381,1
3,215,129,1
4,285,450,1


In [327]:
t2 = feature2[:][['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!='null')&(t2.date_received!='null')][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)
t2.head()

,user_id,merchant_id,user_merchant_buy_use_coupon
0,165,2934,4
1,165,4195,4
2,184,3381,1
3,215,129,1
4,285,450,1


In [328]:
t3 = feature2[:][['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
t3.head()


,user_id,merchant_id,user_merchant_any
0,165,2934,4
1,165,4195,4
2,184,3381,1
3,215,129,1
4,285,450,1


In [329]:
t4 = feature2[:][['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date!='null')&(t4.coupon_id=='null')][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)
t4.head()

,user_id,merchant_id,user_merchant_buy_common


In [330]:
user_merchant2 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t1,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t2,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t3,on=['user_id','merchant_id'],how='left')
user_merchant2 = pd.merge(user_merchant2,t4,on=['user_id','merchant_id'],how='left')
user_merchant2.user_merchant_buy_use_coupon = user_merchant2.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant2.user_merchant_buy_common = user_merchant2.user_merchant_buy_common.replace(np.nan,0)
user_merchant2['user_merchant_coupon_transfer_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float') / user_merchant2.user_merchant_received.astype('float')
user_merchant2['user_merchant_coupon_buy_rate'] = user_merchant2.user_merchant_buy_use_coupon.astype('float') / user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2['user_merchant_rate'] = user_merchant2.user_merchant_buy_total.astype('float') / user_merchant2.user_merchant_any.astype('float')
user_merchant2['user_merchant_common_buy_rate'] = user_merchant2.user_merchant_buy_common.astype('float') / user_merchant2.user_merchant_buy_total.astype('float')
user_merchant2.to_csv('user_merchant2.csv',index=None)


In [331]:
user_merchant2.head()

,user_id,merchant_id,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate
0,1439408,2632,1,1,1,1,0.0,1.0,1.0,1.0,0.0
1,253750,8390,1,1,1,1,0.0,1.0,1.0,1.0,0.0
2,2881376,8390,2,2,2,2,0.0,1.0,1.0,1.0,0.0
3,7073472,2934,1,1,1,1,0.0,1.0,1.0,1.0,0.0
4,114747,2454,3,3,3,3,0.0,1.0,1.0,1.0,0.0


In [332]:
user_merchant2.shape

(188470, 11)

# for dataset1

In [333]:
all_user_merchant = feature1[:][['user_id','merchant_id']]
all_user_merchant.drop_duplicates(inplace=True)

In [334]:
t = feature1[:][['user_id','merchant_id','date']]
t = t[t.date!='null'][['user_id','merchant_id']]
t['user_merchant_buy_total'] = 1
t = t.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t.drop_duplicates(inplace=True)


In [335]:
t1 = feature1[:][['user_id','merchant_id','coupon_id']]
t1 = t1[t1.coupon_id!='null'][['user_id','merchant_id']]
t1['user_merchant_received'] = 1
t1 = t1.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t1.drop_duplicates(inplace=True)
t1.head()


,user_id,merchant_id,user_merchant_received
0,165,2934,4
1,165,4195,2
2,184,3381,1
3,215,129,1
4,285,450,1


In [336]:
t2 = feature1[:][['user_id','merchant_id','date','date_received']]
t2 = t2[(t2.date!='null')&(t2.date_received!='null')][['user_id','merchant_id']]
t2['user_merchant_buy_use_coupon'] = 1
# print(t2.dtypes)
# print(t2.reset_index())
#t2 = t2.reset_index().groupby(['user_id','merchant_id']).agg('sum')
t2 = t2.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t2.drop_duplicates(inplace=True)
t2.head()

,user_id,merchant_id,user_merchant_buy_use_coupon
0,165,2934,4
1,165,4195,2
2,184,3381,1
3,215,129,1
4,285,450,1


In [337]:
t3 = feature1[:][['user_id','merchant_id']]
t3['user_merchant_any'] = 1
t3 = t3.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t3.drop_duplicates(inplace=True)
t3.head()


,user_id,merchant_id,user_merchant_any
0,165,2934,4
1,165,4195,2
2,184,3381,1
3,215,129,1
4,285,450,1


In [338]:
t4 = feature1[:][['user_id','merchant_id','date','coupon_id']]
t4 = t4[(t4.date!='null')&(t4.coupon_id=='null')][['user_id','merchant_id']]
t4['user_merchant_buy_common'] = 1
t4 = t4.groupby(['user_id','merchant_id']).agg('sum').reset_index()
t4.drop_duplicates(inplace=True)
t4.head()

,user_id,merchant_id,user_merchant_buy_common


In [339]:
user_merchant1 = pd.merge(all_user_merchant,t,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t1,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t2,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t3,on=['user_id','merchant_id'],how='left')
user_merchant1 = pd.merge(user_merchant1,t4,on=['user_id','merchant_id'],how='left')
user_merchant1.user_merchant_buy_use_coupon = user_merchant1.user_merchant_buy_use_coupon.replace(np.nan,0)
user_merchant1.user_merchant_buy_common = user_merchant1.user_merchant_buy_common.replace(np.nan,0)
user_merchant1['user_merchant_coupon_transfer_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float') / user_merchant1.user_merchant_received.astype('float')
user_merchant1['user_merchant_coupon_buy_rate'] = user_merchant1.user_merchant_buy_use_coupon.astype('float') / user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1['user_merchant_rate'] = user_merchant1.user_merchant_buy_total.astype('float') / user_merchant1.user_merchant_any.astype('float')
user_merchant1['user_merchant_common_buy_rate'] = user_merchant1.user_merchant_buy_common.astype('float') / user_merchant1.user_merchant_buy_total.astype('float')
user_merchant1.to_csv('user_merchant1.csv',index=None)


In [340]:
user_merchant1.head()

,user_id,merchant_id,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate
0,1439408,2632,1,1,1,1,0.0,1.0,1.0,1.0,0.0
1,253750,8390,1,1,1,1,0.0,1.0,1.0,1.0,0.0
2,1964720,7884,1,1,1,1,0.0,1.0,1.0,1.0,0.0
3,1113008,1041,1,1,1,1,0.0,1.0,1.0,1.0,0.0
4,2881376,8390,2,2,2,2,0.0,1.0,1.0,1.0,0.0


In [341]:
user_merchant1.shape

(169418, 11)

#  generate training and testing set ################

In [342]:
def get_label(s):
    s = s.split(':')
    if s[0]=='null':
        return 0
    elif (date(int(s[0][0:4]),int(s[0][4:6]),int(s[0][6:8]))-date(int(s[1][0:4]),int(s[1][4:6]),int(s[1][6:8]))).days<=15:
        return 1
    else:
        return -1

In [343]:
coupon3 = pd.read_csv('coupon3_feature.csv')
merchant3 = pd.read_csv('merchant3_feature.csv')
user3 = pd.read_csv('user3_feature.csv')
user_merchant3 = pd.read_csv('user_merchant3.csv')
other_feature3 = pd.read_csv('other_feature3.csv')
dataset3 = pd.merge(coupon3,merchant3,on='merchant_id',how='left')
dataset3 = pd.merge(dataset3,user3,on='user_id',how='left')
dataset3 = pd.merge(dataset3,user_merchant3,on=['user_id','merchant_id'],how='left')
dataset3 = pd.merge(dataset3,other_feature3,on=['user_id','coupon_id','date_received'],how='left')
dataset3.drop_duplicates(inplace=True)


In [344]:
dataset3.user_merchant_buy_total = dataset3.user_merchant_buy_total.replace(np.nan,0)
dataset3.user_merchant_any = dataset3.user_merchant_any.replace(np.nan,0)
dataset3.user_merchant_received = dataset3.user_merchant_received.replace(np.nan,0)
dataset3['is_weekend'] = dataset3.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset3.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset3 = pd.concat([dataset3,weekday_dummies],axis=1)

#dataset3.drop(['merchant_id','day_of_week'],axis=1,inplace=True)
dataset3.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)

#dataset3.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)
dataset3 = dataset3.replace('null',np.nan)
dataset3.to_csv('dataset3.csv',index=None)



In [345]:
pd.set_option('display.max_columns',200)
dataset3.head()

,user_id,coupon_id,discount_rate,distance,date_received,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,4129537,9983,0.833333,0.0,20160712,12,12,30.0,5.0,1,7757.0,7757.0,7757.0,-1.0,10.0,0.916076,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0
1,6949378,3429,0.833333,1.0,20160706,6,6,30.0,5.0,1,119.0,119.0,119.0,-1.0,10.0,1.613445,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,3.0,3.0,3.0,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,1,0,0,0,0
2,2166529,6928,0.900000,0.0,20160727,27,27,200.0,20.0,1,2280.0,2280.0,2280.0,-1.0,10.0,2.235088,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,2,1,-1,-1,0,0,0,1,0,0,0,0
3,2166529,1808,0.900000,0.0,20160727,27,27,100.0,10.0,1,2280.0,2280.0,2280.0,-1.0,10.0,2.235088,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,2,1,-1,-1,0,0,0,1,0,0,0,0
4,6172162,6500,0.966667,0.0,20160708,8,8,30.0,1.0,1,1065.0,1065.0,1065.0,-1.0,10.0,1.094836,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0


In [346]:
dataset3.shape

(113234, 57)

In [347]:
coupon2 = pd.read_csv('coupon2_feature.csv')
merchant2 = pd.read_csv('merchant2_feature.csv')
user2 = pd.read_csv('user2_feature.csv')
user_merchant2 = pd.read_csv('user_merchant2.csv')
other_feature2 = pd.read_csv('other_feature2.csv')
dataset2 = pd.merge(coupon2,merchant2,on='merchant_id',how='left')
dataset2 = pd.merge(dataset2,user2,on='user_id',how='left')
dataset2 = pd.merge(dataset2,user_merchant2,on=['user_id','merchant_id'],how='left')
dataset2 = pd.merge(dataset2,other_feature2,on=['user_id','coupon_id','date_received'],how='left')
dataset2.drop_duplicates(inplace=True)
dataset2.shape


(257126, 53)

In [348]:
pd.set_option('display.max_columns',200)   #解决输出省略的问题
dataset2.user_merchant_buy_total = dataset2.user_merchant_buy_total.replace(np.nan,0)
dataset2.user_merchant_any = dataset2.user_merchant_any.replace(np.nan,0)
dataset2.user_merchant_received = dataset2.user_merchant_received.replace(np.nan,0)
dataset2['is_weekend'] = dataset2.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset2.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset2 = pd.concat([dataset2,weekday_dummies],axis=1)

dataset2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,1439408,4663,11002.0,0.866667,1.0,20160528,NaN,6,28,14,150.0,20.0,1,7730,657.0,657.0,657.0,-1.0,10.0,2.619482,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,3,-1,-1,1,1,-1,-1,1,0,0,0,0,0,1,0
1,1439408,2632,8591.0,0.950000,0.0,20160613,NaN,1,13,30,20.0,1.0,1,5,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,1,0,1,1,28,-1,0,1,0,0,0,0,0,0
2,1439408,2632,8591.0,0.950000,0.0,20160516,20160613.0,1,16,2,20.0,1.0,1,5,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,0,1,1,1,-1,28,0,1,0,0,0,0,0,0
3,2029232,450,1532.0,0.833333,0.0,20160530,NaN,1,30,16,30.0,5.0,1,11728,6307.0,6307.0,6307.0,-1.0,10.0,0.789282,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,1,0,0,0,0,0,0
4,2029232,6459,12737.0,0.950000,0.0,20160519,NaN,4,19,5,20.0,1.0,1,16,5.0,5.0,5.0,0.0,1.0,0.800000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0


In [239]:
# dataset2.drop(['merchant_id','day_of_week'],axis=1,inplace=True)
# #dataset3.drop(['merchant_id','day_of_week','coupon_count'],axis=1,inplace=True)
# dataset2 = dataset3.replace('null',np.nan)
# dataset2.to_csv('dataset2.csv',index=None)

In [349]:
dataset2.shape

(257126, 61)

In [350]:
dataset2.head()

,user_id,merchant_id,coupon_id,discount_rate,distance,date_received,date,day_of_week,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,coupon_count,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7
0,1439408,4663,11002.0,0.866667,1.0,20160528,NaN,6,28,14,150.0,20.0,1,7730,657.0,657.0,657.0,-1.0,10.0,2.619482,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,3,-1,-1,1,1,-1,-1,1,0,0,0,0,0,1,0
1,1439408,2632,8591.0,0.950000,0.0,20160613,NaN,1,13,30,20.0,1.0,1,5,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,1,0,1,1,28,-1,0,1,0,0,0,0,0,0
2,1439408,2632,8591.0,0.950000,0.0,20160516,20160613.0,1,16,2,20.0,1.0,1,5,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,0,1,1,1,-1,28,0,1,0,0,0,0,0,0
3,2029232,450,1532.0,0.833333,0.0,20160530,NaN,1,30,16,30.0,5.0,1,11728,6307.0,6307.0,6307.0,-1.0,10.0,0.789282,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,1,0,0,0,0,0,0
4,2029232,6459,12737.0,0.950000,0.0,20160519,NaN,4,19,5,20.0,1.0,1,16,5.0,5.0,5.0,0.0,1.0,0.800000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0


In [351]:
#运用函数提取label
dataset2.date = dataset2.date.replace(np.nan,'null')
dataset2['label'] = dataset2.date.astype('str') + ':' +  dataset2.date_received.astype('str')
dataset2.label = dataset2.label.apply(get_label)
dataset2.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset2.head()


,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1439408,0.866667,1.0,28,14,150.0,20.0,1,657.0,657.0,657.0,-1.0,10.0,2.619482,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,3,-1,-1,1,1,-1,-1,1,0,0,0,0,0,1,0,0
1,1439408,0.950000,0.0,13,30,20.0,1.0,1,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,1,0,1,1,28,-1,0,1,0,0,0,0,0,0,0
2,1439408,0.950000,0.0,16,2,20.0,1.0,1,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,0,1,1,1,-1,28,0,1,0,0,0,0,0,0,-1
3,2029232,0.833333,0.0,30,16,30.0,5.0,1,6307.0,6307.0,6307.0,-1.0,10.0,0.789282,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,1,0,0,0,0,0,0,0
4,2029232,0.950000,0.0,19,5,20.0,1.0,1,5.0,5.0,5.0,0.0,1.0,0.800000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0,0


In [352]:
dataset2.shape

(257126, 56)

In [353]:
dataset2 = dataset2.replace('null',np.nan)
dataset2.to_csv('dataset2.csv',index=None)
dataset2.head()

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1439408,0.866667,1.0,28,14,150.0,20.0,1,657.0,657.0,657.0,-1.0,10.0,2.619482,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,3,-1,-1,1,1,-1,-1,1,0,0,0,0,0,1,0,0
1,1439408,0.950000,0.0,13,30,20.0,1.0,1,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,1,0,1,1,28,-1,0,1,0,0,0,0,0,0,0
2,1439408,0.950000,0.0,16,2,20.0,1.0,1,14.0,14.0,14.0,0.0,10.0,2.071429,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,2,3,0,1,1,1,-1,28,0,1,0,0,0,0,0,0,-1
3,2029232,0.833333,0.0,30,16,30.0,5.0,1,6307.0,6307.0,6307.0,-1.0,10.0,0.789282,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,1,0,0,0,0,0,0,0
4,2029232,0.950000,0.0,19,5,20.0,1.0,1,5.0,5.0,5.0,0.0,1.0,0.800000,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,2,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0,0


In [354]:
coupon1 = pd.read_csv('coupon1_feature.csv')
merchant1 = pd.read_csv('merchant1_feature.csv')
user1 = pd.read_csv('user1_feature.csv')
user_merchant1 = pd.read_csv('user_merchant1.csv')
other_feature1 = pd.read_csv('other_feature1.csv')
dataset1 = pd.merge(coupon1,merchant1,on='merchant_id',how='left')
dataset1 = pd.merge(dataset1,user1,on='user_id',how='left')
dataset1 = pd.merge(dataset1,user_merchant1,on=['user_id','merchant_id'],how='left')
dataset1 = pd.merge(dataset1,other_feature1,on=['user_id','coupon_id','date_received'],how='left')
dataset1.drop_duplicates(inplace=True)
dataset1.shape

(136301, 53)

In [355]:
pd.set_option('display.max_columns',200)   #解决输出省略的问题
dataset1.user_merchant_buy_total = dataset1.user_merchant_buy_total.replace(np.nan,0)
dataset1.user_merchant_any = dataset1.user_merchant_any.replace(np.nan,0)
dataset1.user_merchant_received = dataset1.user_merchant_received.replace(np.nan,0)
dataset1['is_weekend'] = dataset1.day_of_week.apply(lambda x:1 if x in (6,7) else 0)
weekday_dummies = pd.get_dummies(dataset1.day_of_week)
weekday_dummies.columns = ['weekday'+str(i+1) for i in range(weekday_dummies.shape[1])]
dataset1 = pd.concat([dataset1,weekday_dummies],axis=1)


In [356]:
#运用函数提取label
dataset1.date = dataset1.date.replace(np.nan,'null')
dataset1['label'] = dataset1.date.astype('str') + ':' +  dataset1.date_received.astype('str')
dataset1.label = dataset1.label.apply(get_label)
dataset1.drop(['merchant_id','day_of_week','date','date_received','coupon_id','coupon_count'],axis=1,inplace=True)
dataset1 = dataset1.replace('null',np.nan)
dataset1.to_csv('dataset1.csv',index=None)

In [357]:
dataset1.head()

,user_id,discount_rate,distance,day_of_month,days_distance,discount_man,discount_jian,is_man_jian,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,merchant_coupon_transfer_rate,coupon_rate,count_merchant,user_min_distance,user_max_distance,user_mean_distance,user_median_distance,buy_use_coupon,buy_total,coupon_received,avg_user_date_datereceived_gap,min_user_date_datereceived_gap,max_user_date_datereceived_gap,buy_use_coupon_rate,user_coupon_transfer_rate,user_merchant_buy_total,user_merchant_received,user_merchant_buy_use_coupon,user_merchant_any,user_merchant_buy_common,user_merchant_coupon_transfer_rate,user_merchant_coupon_buy_rate,user_merchant_rate,user_merchant_common_buy_rate,this_month_user_receive_same_coupon_count,this_month_user_receive_all_coupon_count,this_month_user_receive_same_coupon_lastone,this_month_user_receive_same_coupon_firstone,this_day_user_receive_all_coupon_count,this_day_user_receive_same_coupon_count,day_gap_before,day_gap_after,is_weekend,weekday1,weekday2,weekday3,weekday4,weekday5,weekday6,weekday7,label
0,1832624,0.900000,0.0,29,16,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
1,163606,0.850000,10.0,21,8,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,1,0,0,0,0
2,4061024,0.900000,10.0,26,13,200.0,20.0,1,14962.0,14962.0,14962.0,-1.0,10.0,1.209798,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0
3,106443,0.833333,-1.0,29,16,30.0,5.0,1,6553.0,6553.0,6553.0,-1.0,10.0,0.664123,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,0,0,0,1,0,0,0
4,114747,0.850000,9.0,26,13,200.0,30.0,1,787.0,787.0,787.0,-1.0,10.0,2.439644,1.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,11.0,11.0,11.0,13.4,1.0,25.0,1.0,1.0,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,1,1,-1,-1,1,1,-1,-1,0,0,1,0,0,0,0,0,0


In [358]:
dataset1.shape

(136301, 56)